In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import sys

sys.path.append("../data")
import pred_fev1
import breathe_data
import effort_corrected_fev1

sys.path.append("../milestone_model")
import lung_health_models

plotsdir = "../../../../PlotsSmartcare/O2_FEV1/"

In [2]:
df = breathe_data.build_O2_FEV1_df()


*** Building O2 Saturation and FEV1 dataframe ***

*** Loading patients data ***
Warning - 4 NaN values in dataframe
The 4 NaN values belong to IDs ('322', '338', '344', '348') whose height are missing.
However, we don't correct for them as we don't have any measurement corresponding to those IDs for now.
Loaded 258 individuals

*** Loading measurements data ***
Dropping 1 entries with FEV1 = 6.0 for ID 330
* Checking for same day measurements *
* Checking for same day measurements *
Number of IDs:  233
Number of rows:  26812
Number of FEV1 recordings: 23778
Number of O2 Saturation recordings: 23431
Dropped 0 entries with NaN O2 Saturation and NaN FEV1
Dropped 3381 entries with NaN O2 Saturation
Dropped 3034 entries with NaN FEV1
20397 entries remain


/Users/tristan.trebaol/Desktop/PhD/Code/phd/src/O2_modelling/../data/effort_corrected_fev1.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ecFEV1"][mask] = smooth.smooth_vector(df.FEV1[mask].to_numpy(), "max")
/Users/tristan.trebaol/Desktop/PhD/Code/phd/src/O2_modelling/../data/effort_corrected_fev1.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ecFEV1"][mask] = smooth.smooth_vector(df.FEV1[mask].to_numpy(), "max")
/Users/tristan.trebaol/Desktop/PhD/Code/phd/src/O2_modelling/../data/effort_corrected_fev1.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

Warning for ID 202: FEV1 % Predicted should be in 0-140%, got 147.24696075623552
Warning for ID 202: FEV1 % Predicted should be in 0-140%, got 147.24696075623552
Warning for ID 202: FEV1 % Predicted should be in 0-140%, got 149.50534972488947
Warning for ID 202: FEV1 % Predicted should be in 0-140%, got 149.50534972488947
Warning for ID 202: FEV1 % Predicted should be in 0-140%, got 149.50534972488947
Warning for ID 202: FEV1 % Predicted should be in 0-140%, got 147.24696075623552
Warning for ID 202: FEV1 % Predicted should be in 0-140%, got 149.50534972488947
Built data structure with 213 IDs and 20397 entries


In [3]:
df.head()

,ID,Date Recorded,FEV1,O2 Saturation,ecFEV1,Age,Sex,Height,Predicted FEV1,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy
0,101,2019-02-20,1.31,97.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,36.287474,99.767593
1,101,2019-02-21,1.29,96.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,35.733466,98.739061
2,101,2019-02-22,1.32,96.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,36.564477,98.739061
3,101,2019-02-23,1.28,97.0,1.33,53,Male,173.0,3.610061,97.22596,36.841481,35.456463,99.767593
4,101,2019-02-24,1.33,98.0,1.36,53,Male,173.0,3.610061,97.22596,37.672492,36.841481,100.796125


In [3]:
df.describe()

,FEV1,O2 Saturation,ecFEV1,Age,Height,Predicted FEV1,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy
count,20397.000000,20397.000000,20397.000000,20397.000000,20397.000000,20397.000000,20397.000000,20397.000000,20397.000000,20397.000000
mean,2.198363,96.966501,2.263423,34.801147,166.247120,3.507772,97.711339,64.659185,62.787907,99.238755
std,0.816148,1.649808,0.822063,10.154773,9.151066,0.649323,0.449598,20.301483,20.271387,1.686950
min,0.490000,76.000000,0.500000,18.000000,143.000000,2.213968,96.975001,15.320382,15.013975,77.519654
25%,1.550000,96.000000,1.610000,27.000000,160.000000,2.979444,97.225960,48.198629,46.536607,98.699543
50%,2.030000,97.000000,2.090000,34.000000,166.000000,3.386997,97.989462,62.880920,61.235539,99.767593
75%,2.760000,98.000000,2.830000,41.000000,173.000000,3.987357,98.114941,77.657779,75.719884,100.255981
max,5.260000,100.000000,5.260000,64.000000,193.000000,5.322753,98.340804,149.505350,149.505350,103.026043


## Factor - Airway resistance vs O2 drop

In [12]:
# Infer airway resistance using the model

def infer_airway_resistance_for_ID(df_for_ID):
    print(len(df_for_ID))
    # TODO: update predFEV1 sigma!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    healthy_FEV1_prior = {"type": "gaussian", "mu": df_for_ID["Predicted FEV1"], "sigma": 0.4}
    model, FEV1, HFEV1, prior_u, AR, prior_ar = lung_health_models.build_HFEV1_AR_FEV1_model(healthy_FEV1_prior)
    for FEV1_obs in df_for_ID.FEV1:
        res_ar = lung_health_models.infer(model, [AR], [[FEV1, FEV1_obs]])
        # Take most probably airway resistance
        most_probable_airway_resistance = max(res_ar.values)
        print(most_probable_airway_resistance)
        return most_probable_airway_resistance

df.groupby(["ID"]).apply(infer_airway_resistance_for_ID)

816
*** Building lung model with HFEV1 and AB ***
816
*** Building lung model with HFEV1 and AB ***


TypeError: unsupported format string passed to Series.__format__

In [7]:
# # 0% airway resistance: FEV1 = Predicted FEV1
# # 25% airway resistance: FEV1 = 0.75 * Predicted FEV1
# # Negative airway resistance: FEV1 > Predicted FEV1
# # Airway resistance = 1 - FEV1/Predicted FEV1
# df["Airway Resistance (%)"] = 100 - df["ecFEV1 % Predicted"]

df["O2 Sat / Healthy O2 Sat (%)"] = (
    df["O2 Saturation"] / df["Healthy O2 Saturation"]
) * 100

df.head()

,ID,Date Recorded,FEV1,O2 Saturation,ecFEV1,Age,Sex,Height,Predicted FEV1,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy,Airway Resistance (%),O2 Sat / Healthy O2 Sat (%)
0,101,2019-02-20,1.31,97.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,36.287474,99.767593,63.435523,99.767593
1,101,2019-02-21,1.29,96.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,35.733466,98.739061,63.435523,98.739061
2,101,2019-02-22,1.32,96.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,36.564477,98.739061,63.435523,98.739061
3,101,2019-02-23,1.28,97.0,1.33,53,Male,173.0,3.610061,97.22596,36.841481,35.456463,99.767593,63.158519,99.767593
4,101,2019-02-24,1.33,98.0,1.36,53,Male,173.0,3.610061,97.22596,37.672492,36.841481,100.796125,62.327508,100.796125


### Raw scatter plot

In [8]:
# Plot Airway resistance vs O2 drop
title = f"Airway Resistance vs O2 drop ({df.ID.nunique()} IDs, {len(df)} datapoints)"
fig = px.scatter(
    df, x="Airway Resistance (%)", y="O2 Sat / Healthy O2 Sat (%)", title=title
)
# Reduce marker size
fig.update_traces(marker=dict(size=2))
fig.show()
# 100% is about 98 for females and 97.4 for males
# Hence, threshold at 95% => 3-3.5% drop in O2 Saturation => 96.5-97% O2 Saturation

### Specific cases

In [18]:
# Plot FEV1 % Predicted with time for individual 122
def plot_fev1_o2(df, ids):
    for id in ids:
        df_for_ID = df[df.ID == id]
        # Create subplot with 2 rows
        fig = make_subplots(rows=2, cols=1)
        # Add trace for FEV1 % Predicted on one subplot
        fig.add_trace(
            go.Scatter(
                x=df_for_ID["Date Recorded"],
                y=df_for_ID["ecFEV1 % Predicted"],
                mode="markers",
                name="ecFEV1 % Predicted",
            ),
            row=1,
            col=1,
        )
        # fig.add_trace(
        #     go.Scatter(
        #         x=df_for_ID["Date Recorded"],
        #         y=df_for_ID["FEV1 % Predicted"],
        #         mode="markers",
        #         name="FEV1 % Predicted",
        #     ),
        #     row=1,
        #     col=1,
        # )
        # Add trace for O2 Saturation on another subplot
        fig.add_trace(
            go.Scatter(
                x=df_for_ID["Date Recorded"],
                y=df_for_ID["O2 Saturation"],
                mode="markers",
                name="O2 Saturation",
            ),
            row=2,
            col=1,
        )
        fig.update_traces(marker=dict(size=3), line=dict(width=0.5))
        title = f"ecFEV1 % Predicted and O2 Sat for individual {id} ({len(df_for_ID)} datapoints)"
        fig.update_layout(title=title)
        # Add trace for O2 Saturation on another subplot

        fig.show()

#### Low airway resistance

In [9]:
# Filter airway resistance below 40%
df[df["Airway Resistance (%)"] < -20]
# '113', '126', '202', '331'

,ID,Date Recorded,FEV1,O2 Saturation,ecFEV1,Age,Sex,Height,Predicted FEV1,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy,Airway Resistance (%),O2 Sat / Healthy O2 Sat (%)
2782,113,2020-11-23,4.05,98.0,4.92,26,Male,165.5,3.923640,97.320070,125.393781,103.220490,100.698654,-25.393781,100.698654
2783,113,2020-11-24,4.92,98.0,5.19,26,Male,165.5,3.923640,97.320070,132.275147,125.393781,100.698654,-32.275147,100.698654
2784,113,2020-12-07,5.19,99.0,5.19,26,Male,165.5,3.923640,97.320070,132.275147,132.275147,101.726191,-32.275147,101.726191
2785,113,2020-12-08,5.15,98.0,5.19,26,Male,165.5,3.923640,97.320070,132.275147,131.255685,100.698654,-32.275147,100.698654
2786,113,2020-12-09,4.63,99.0,5.15,26,Male,165.5,3.923640,97.320070,131.255685,118.002684,101.726191,-31.255685,101.726191
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13720,202,2021-04-23,2.80,98.0,2.80,64,Female,157.0,2.213968,98.165133,126.469782,126.469782,99.831781,-26.469782,99.831781
13721,202,2021-04-30,1.83,100.0,2.80,64,Female,157.0,2.213968,98.165133,126.469782,82.657036,101.869164,-26.469782,101.869164
20107,331,2021-05-26,2.62,99.0,3.02,54,Female,157.0,2.472585,98.165133,122.139372,105.961972,100.850472,-22.139372,100.850472
20108,331,2021-05-29,3.02,97.0,3.02,54,Female,157.0,2.472585,98.165133,122.139372,122.139372,98.813089,-22.139372,98.813089


In [12]:
plot_fev1_o2(df, ["113", "126", "202", "331"])

#### High airway resistance

In [14]:
# Filter airway resistance below 40%
df[df["Airway Resistance (%)"] > 80]
# 3 individuals '122', '198', '286' have airway resistance > 80%

,ID,Date Recorded,FEV1,O2 Saturation,ecFEV1,Age,Sex,Height,Predicted FEV1,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy,Airway Resistance (%),O2 Sat / Healthy O2 Sat (%)
18436,286,2020-10-02,0.50,90.0,0.50,42,Female,170.0,3.263626,98.00201,15.320382,15.320382,91.834852,84.679618,91.834852
18437,286,2020-10-03,0.50,93.0,0.50,42,Female,170.0,3.263626,98.00201,15.320382,15.320382,94.896013,84.679618,94.896013
18438,286,2020-10-11,0.49,89.0,0.50,42,Female,170.0,3.263626,98.00201,15.320382,15.013975,90.814464,84.679618,90.814464
18439,286,2020-10-17,0.50,90.0,0.50,42,Female,170.0,3.263626,98.00201,15.320382,15.320382,91.834852,84.679618,91.834852
18440,286,2020-10-25,0.49,91.0,0.50,42,Female,170.0,3.263626,98.00201,15.320382,15.013975,92.855239,84.679618,92.855239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18495,286,2021-06-04,0.50,90.0,0.51,42,Female,170.0,3.263626,98.00201,15.626790,15.320382,91.834852,84.373210,91.834852
18496,286,2021-06-12,0.51,90.0,0.51,42,Female,170.0,3.263626,98.00201,15.626790,15.626790,91.834852,84.373210,91.834852
18497,286,2021-06-20,0.50,90.0,0.51,42,Female,170.0,3.263626,98.00201,15.626790,15.320382,91.834852,84.373210,91.834852
18498,286,2021-06-24,0.50,90.0,0.50,42,Female,170.0,3.263626,98.00201,15.320382,15.320382,91.834852,84.679618,91.834852


In [15]:
# Plot FEV1 % Predicted with time for individual 122
plot_fev1_o2(df, ["122", "198", "286"])

#### High O2 Drop

In [16]:
df[df["O2 Sat / Healthy O2 Sat (%)"] < 90].ID.unique()

array(['110', '111', '180', '352'], dtype=object)

In [17]:
plot_fev1_o2(df, ["111", "180", "352"])

### Factor function

In [20]:
# We wanna plot a noise-agnostic version of the raw scatter plot's top envelope

# > 50 datapoints per bin

# Discretise airway resistance
# Take 90th percentile for each bin
# Plot 90th percentile and amount of data per bin

df_aw_o2_factor

,ID,Date Recorded,FEV1,O2 Saturation,ecFEV1,Age,Sex,Height,Predicted FEV1,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy,Airway Resistance (%),O2 Sat / Healthy O2 Sat (%)
0,101,2019-02-20,1.31,97.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,36.287474,99.767593,63.435523,99.767593
1,101,2019-02-21,1.29,96.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,35.733466,98.739061,63.435523,98.739061
2,101,2019-02-22,1.32,96.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,36.564477,98.739061,63.435523,98.739061
3,101,2019-02-23,1.28,97.0,1.33,53,Male,173.0,3.610061,97.22596,36.841481,35.456463,99.767593,63.158519,99.767593
4,101,2019-02-24,1.33,98.0,1.36,53,Male,173.0,3.610061,97.22596,37.672492,36.841481,100.796125,62.327508,100.796125
